In [45]:
from aidream_data.core import AidreamDatabase
from aidream_data import constants

from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import shutil
import os


# 0. Load Aidream_database, list of perfusion patients : 

In [46]:
# Load the AIDREAM Database : 
aidream_db = AidreamDatabase()
aidream_db.connect()


In [47]:
# Load the perfusion patients list : 
with open("list_patients.txt", 'r') as f:
    list_patients = f.readlines()

list_patients = [p.strip('\n') for p in list_patients]
    

In [48]:
def sort_name_list(list_patients):

    sorted_list = [list_patients[i] for i in np.argsort([int(p.strip("AIDREAM_")) for p in list_patients])]
    return sorted_list
    

# 1. Load MRI DICOM data, for the native ref : 

In [10]:
# export MRI images in native ref for both pre_RT and Rechute  : 
aidream_dataset = aidream_db.create_dataset(list_aidream_ids=list_patients,
                                            list_pre_rt=['T1', 'T1CE', 'FLAIR'],
                                            list_rechute=['T1', 'T1CE', 'FLAIR'],
                                            load_anyway=True)


In [11]:
# aidream_dataset.export("/media/maichi/T9/AIDREAM_DATA")


# 2. Load PERFUSION data : 

In [16]:
dir_src_perfusion = Path("/media/maichi/SSD-IGR/Perfusion data")
dir_aidream_data = Path("/media/maichi/T9/AIDREAM_DATA")

for patient in tqdm(list_patients):

    try:
        dir_src = dir_src_perfusion / patient 
        
        dir_dst = dir_aidream_data / patient / "pre_RT" / "PERFUSION" / "DICOM"
        dir_dst.parent.mkdir(parents=True, exist_ok=True)

        shutil.copytree(src=dir_src, dst=dir_dst)

    except Exception as e:
        print(fr"{patient} has no PERFUSION data !")
        

  2%|██▊                                                                                                                                                                     | 4/234 [00:05<04:53,  1.27s/it]

AIDREAM_5 has no PERFUSION data !


  9%|███████████████▋                                                                                                                                                       | 22/234 [00:27<04:40,  1.32s/it]

AIDREAM_25 has no PERFUSION data !


 11%|██████████████████▌                                                                                                                                                    | 26/234 [00:31<04:04,  1.17s/it]

...

 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎     | 226/234 [04:22<00:11,  1.45s/it]

AIDREAM_389 has no PERFUSION data !


 99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊  | 231/234 [04:28<00:04,  1.45s/it]

AIDREAM_397 has no PERFUSION data !


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [04:31<00:00,  1.16s/it]


In [49]:
list_patients_interim = os.listdir(dir_src_perfusion)
print(fr"Number of patients with available PERFUSION data (for now) : {len(list_patients_interim)}")


Number of patients with available PERFUSION data (for now) : 198


In [20]:
with open("list_intermediary_patients.txt", "w") as f:
    f.writelines([f"{p}\n" for p in list_patients])
    

# 3. Load the CERCARE Data : 

In [54]:
dir_src_cercare = Path("/media/maichi/T7/AIDREAM_DATA/raw")

list_patients_interim = set(list_patients)


for patient in tqdm(list_patients):
    for imaging in ['CTH', 'OEF', 'rCBV', 'Delay', 'COV', 'rCMRO2', 'rLeakage']:
        
        try:
            dir_src = dir_src_cercare / patient / "pre_RT" / imaging
            
            dir_dst = dir_aidream_data / patient / "pre_RT" / imaging / "DICOM"
            dir_dst.parent.mkdir(parents=True, exist_ok=True)
        
            shutil.copytree(src=dir_src, dst=dir_dst)            
    
        except Exception as e:

            list_patients_interim = list_patients_interim - {patient}
            print(fr"{patient} has no {imaging} data !")   
    

  3%|█████                                                                                                                                                                   | 7/234 [00:00<00:07, 30.99it/s]

AIDREAM_5 has no CTH data !
AIDREAM_5 has no OEF data !
AIDREAM_5 has no rCBV data !
AIDREAM_5 has no Delay data !
AIDREAM_5 has no COV data !
AIDREAM_5 has no rCMRO2 data !
AIDREAM_5 has no rLeakage data !
AIDREAM_10 has no CTH data !
AIDREAM_10 has no OEF data !
AIDREAM_10 has no rCBV data !
AIDREAM_10 has no Delay data !
AIDREAM_10 has no COV data !
AIDREAM_10 has no rCMRO2 data !
AIDREAM_10 has no rLeakage data !


  8%|█████████████▌                                                                                                                                                         | 19/234 [00:00<00:04, 44.40it/s]

AIDREAM_15 has no CTH data !
AIDREAM_15 has no OEF data !
AIDREAM_15 has no rCBV data !
AIDREAM_15 has no Delay data !
AIDREAM_15 has no COV data !
AIDREAM_15 has no rCMRO2 data !
AIDREAM_15 has no rLeakage data !
AIDREAM_16 has no CTH data !
AIDREAM_16 has no OEF data !
AIDREAM_16 has no rCBV data !
AIDREAM_16 has no Delay data !
AIDREAM_16 has no COV data !
AIDREAM_16 has no rCMRO2 data !
AIDREAM_16 has no rLeakage data !
AIDREAM_18 has no CTH data !
AIDREAM_18 has no OEF data !
AIDREAM_18 has no rCBV data !
AIDREAM_18 has no Delay data !
AIDREAM_18 has no COV data !
AIDREAM_18 has no rCMRO2 data !
AIDREAM_18 has no rLeakage data !
AIDREAM_19 has no CTH data !
AIDREAM_19 has no OEF data !
AIDREAM_19 has no rCBV data !
AIDREAM_19 has no Delay data !
AIDREAM_19 has no COV data !
AIDREAM_19 has no rCMRO2 data !
AIDREAM_19 has no rLeakage data !
AIDREAM_25 has no CTH data !
AIDREAM_25 has no OEF data !
AIDREAM_25 has no rCBV data !
AIDREAM_25 has no Delay data !
AIDREAM_25 has no COV dat

 14%|███████████████████████▌                                                                                                                                               | 33/234 [00:00<00:03, 55.14it/s]

...

AIDREAM_226 has no CTH data !
AIDREAM_226 has no OEF data !
AIDREAM_226 has no rCBV data !
AIDREAM_226 has no Delay data !
AIDREAM_226 has no COV data !
AIDREAM_226 has no rCMRO2 data !
AIDREAM_226 has no rLeakage data !
AIDREAM_230 has no CTH data !
AIDREAM_230 has no OEF data !
AIDREAM_230 has no rCBV data !
AIDREAM_230 has no Delay data !
AIDREAM_230 has no COV data !
AIDREAM_230 has no rCMRO2 data !
AIDREAM_230 has no rLeakage data !
AIDREAM_231 has no CTH data !
AIDREAM_231 has no OEF data !
AIDREAM_231 has no rCBV data !
AIDREAM_231 has no Delay data !
AIDREAM_231 has no COV data !
AIDREAM_231 has no rCMRO2 data !
AIDREAM_231 has no rLeakage data !
AIDREAM_234 has no CTH data !
AIDREAM_234 has no OEF data !
AIDREAM_234 has no rCBV data !
AIDREAM_234 has no Delay data !
AIDREAM_234 has no COV data !
AIDREAM_234 has no rCMRO2 data !
AIDREAM_234 has no rLeakage data !
AIDREAM_235 has no CTH data !
AIDREAM_235 has no OEF data !
AIDREAM_235 has no rCBV data !
AIDREAM_235 has no Delay 

 66%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                        | 155/234 [00:03<00:01, 50.71it/s]

AIDREAM_239 has no CTH data !
AIDREAM_239 has no OEF data !
AIDREAM_239 has no rCBV data !
AIDREAM_239 has no Delay data !
AIDREAM_239 has no COV data !
AIDREAM_239 has no rCMRO2 data !
AIDREAM_239 has no rLeakage data !


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:04<00:00, 57.32it/s]

AIDREAM_248 has no CTH data !
AIDREAM_248 has no OEF data !
AIDREAM_248 has no rCBV data !
AIDREAM_248 has no Delay data !
AIDREAM_248 has no COV data !
AIDREAM_248 has no rCMRO2 data !
AIDREAM_248 has no rLeakage data !
AIDREAM_301 has no CTH data !
AIDREAM_301 has no OEF data !
AIDREAM_301 has no rCBV data !
AIDREAM_301 has no Delay data !
AIDREAM_301 has no COV data !
AIDREAM_301 has no rCMRO2 data !
AIDREAM_301 has no rLeakage data !
AIDREAM_303 has no CTH data !
AIDREAM_303 has no OEF data !
AIDREAM_303 has no rCBV data !
AIDREAM_303 has no Delay data !
AIDREAM_303 has no COV data !
AIDREAM_303 has no rCMRO2 data !
AIDREAM_303 has no rLeakage data !
AIDREAM_306 has no CTH data !
AIDREAM_306 has no OEF data !
AIDREAM_306 has no rCBV data !
AIDREAM_306 has no Delay data !
AIDREAM_306 has no COV data !
AIDREAM_306 has no rCMRO2 data !
AIDREAM_306 has no rLeakage data !
AIDREAM_307 has no CTH data !
AIDREAM_307 has no OEF data !
AIDREAM_307 has no rCBV data !
AIDREAM_307 has no Delay 

In [55]:
list_patients_interim = list(set(list_patients_interim)) 
list_patients_interim = sort_name_list(list_patients_interim)

print(fr"Number of patients with available CERCARE data (for now) : {len(list_patients_interim)}")


Number of patients with available CERCARE data (for now) : 107


In [56]:
with open("list_intermediary_patients.txt", "w") as f:
    f.writelines([f"{p}\n" for p in list_patients_interim])
    

# 4. Load the Labels data : 

In [98]:
remaining_patients = set()

## 4.1. Load the pre_RT Labels data :  

In [99]:
dir_src_prert_labels = constants.DIR_AIDREAM_ON_NAS / 'RSTRUCT_Export2024' / 'DONNEES_VALIDEES' / 'RTSTRUCT_IRM_preRT_09022024'

for patient in tqdm(list_patients):

    try:
        dir_src = dir_src_prert_labels / patient / "RS"

        dir_dst = dir_aidream_data / patient / "pre_RT" / 'Labels' / 'DICOM'
        dir_dst.parent.mkdir(exist_ok=True, parents=True)

        if dir_dst.exists():
            shutil.rmtree(dir_dst)

        shutil.copytree(dir_src, dir_dst)

    except Exception as e:
        print(fr'Failed to load pre_RT Labels for patient {patient} !')
        remaining_patients.add(patient)
        

  6%|██████████▋                                                                                                                                                            | 15/234 [00:00<00:05, 43.49it/s]

Failed to load pre_RT Labels for patient AIDREAM_10 !
Failed to load pre_RT Labels for patient AIDREAM_15 !


 12%|████████████████████▋                                                                                                                                                  | 29/234 [00:00<00:05, 38.56it/s]

Failed to load pre_RT Labels for patient AIDREAM_28 !
Failed to load pre_RT Labels for patient AIDREAM_31 !
Failed to load pre_RT Labels for patient AIDREAM_34 !


 15%|████████████████████████▎                                                                                                                                              | 34/234 [00:00<00:05, 39.53it/s]

...

Failed to load pre_RT Labels for patient AIDREAM_379 !


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  | 230/234 [00:06<00:00, 27.71it/s]

Failed to load pre_RT Labels for patient AIDREAM_390 !


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:06<00:00, 36.75it/s]

Failed to load pre_RT Labels for patient AIDREAM_399 !


In [100]:
len(remaining_patients)

53

In [101]:
dir_src_prert_labels = constants.DIR_AIDREAM_ON_NAS / 'Export_RayStation_RS_27062024' / 'preRT'

for folder in tqdm(os.listdir(dir_src_prert_labels)):

    patient = folder.split("__")[-1]

    try:
        dir_src = dir_src_prert_labels / folder

        dir_dst = dir_aidream_data / patient / "pre_RT" / 'Labels' / 'DICOM'
        dir_dst.parent.mkdir(exist_ok=True, parents=True)

        if dir_dst.exists():

            print(fr"Replacing pre_RT labels for patient {patient}")
            shutil.rmtree(dir_dst)

        else:
            print(fr"Adding pre_RT labels for patient {patient}+")

        shutil.copytree(dir_src, dir_dst)
        remaining_patients = remaining_patients - {patient}
        print(fr"Successfully loaded pre_RT labels for patient {patient}")
        
    except Exception as e:
        print(e)
        print(fr'Failed to load pre_RT Labels for patient {patient} !')
        

 14%|███████████████████████                                                                                                                                                  | 3/22 [00:00<00:00, 28.41it/s]

Replacing pre_RT labels for patient AIDREAM_114
Successfully loaded pre_RT labels for patient AIDREAM_114
Replacing pre_RT labels for patient AIDREAM_135
Successfully loaded pre_RT labels for patient AIDREAM_135
Replacing pre_RT labels for patient AIDREAM_137
Successfully loaded pre_RT labels for patient AIDREAM_137
Replacing pre_RT labels for patient AIDREAM_130
Successfully loaded pre_RT labels for patient AIDREAM_130
Adding pre_RT labels for patient AIDREAM_116+
Successfully loaded pre_RT labels for patient AIDREAM_116
Replacing pre_RT labels for patient AIDREAM_12


 27%|██████████████████████████████████████████████                                                                                                                           | 6/22 [00:00<00:00, 28.00it/s]

Successfully loaded pre_RT labels for patient AIDREAM_12
Replacing pre_RT labels for patient AIDREAM_1
Successfully loaded pre_RT labels for patient AIDREAM_1
Adding pre_RT labels for patient AIDREAM_102+
Successfully loaded pre_RT labels for patient AIDREAM_102
Replacing pre_RT labels for patient AIDREAM_122
Successfully loaded pre_RT labels for patient AIDREAM_122
Replacing pre_RT labels for patient AIDREAM_109


 45%|████████████████████████████████████████████████████████████████████████████▎                                                                                           | 10/22 [00:00<00:00, 29.86it/s]

...

Successfully loaded pre_RT labels for patient AIDREAM_11
Replacing pre_RT labels for patient AIDREAM_13
Successfully loaded pre_RT labels for patient AIDREAM_13
Replacing pre_RT labels for patient AIDREAM_124
Successfully loaded pre_RT labels for patient AIDREAM_124
Replacing pre_RT labels for patient AIDREAM_117
Successfully loaded pre_RT labels for patient AIDREAM_117
Replacing pre_RT labels for patient AIDREAM_101
Successfully loaded pre_RT labels for patient AIDREAM_101
Replacing pre_RT labels for patient AIDREAM_120


 82%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                              | 18/22 [00:00<00:00, 32.87it/s]

Successfully loaded pre_RT labels for patient AIDREAM_120
Replacing pre_RT labels for patient AIDREAM_121
Successfully loaded pre_RT labels for patient AIDREAM_121
Adding pre_RT labels for patient AIDREAM_10+
Successfully loaded pre_RT labels for patient AIDREAM_10
Replacing pre_RT labels for patient AIDREAM_331


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 29.96it/s]

Successfully loaded pre_RT labels for patient AIDREAM_331
Replacing pre_RT labels for patient AIDREAM_115
Successfully loaded pre_RT labels for patient AIDREAM_115


In [102]:
len(remaining_patients)

50